In [ ]:
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt

from glob import glob
from pathlib import Path

# make plots bigger
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 200

This notebook combines the LDV, MDV, HDV, rail, ship, and plane transportation loads into a single file per Balancing Authority.

Notes:
* LDV data is provided for different charging strategies, combined in different proportions based on the year. For example, in 2025, 10% of LDV load comes from the load_leveling charging strategy while 90% comes from the min_delay charging strategy.
* The rail, ship, and plane loads are held constant across the year and are calculated by population weighting county share against the GCAM state level loads.
* County population projections by SSP provided by TELL are redistributed as part of this repository in the input folder

<br/>

In [ ]:
# root path to prepend to the following paths
base_path = '.'

In [ ]:
# paths to the various transportation load datasets

# LDV
path_to_ldv_output = f'{base_path}/LDV/output'

# MDV and HDV
path_to_mhdv_output = f'{base_path}/MHDV/output'

# rail, aviation, and ship
path_to_rail_aviation_ship_data = f'{base_path}/input/rail_aviation_ship_county_share.xlsx'

# path to the file from TELL that maps County FIPS code to Balancing Authority
path_to_county_ba_mapping = f'{base_path}/input/ba_service_territory_2019.csv'

# path to the folder from TELL that contains county populations by year
path_to_county_population = f'{base_path}/input'

# path to the folder containing the GCAM energy output that is generated in the LDV notebook
path_to_gcam_energy_output = f'{base_path}/input'


In [ ]:
# breakdown of how to combine the LDV charging strategies by year
LDV_CHARGING_SHARE_BY_YEAR = {
    2025: {
        'min_delay': 0.9,
        'load_leveling': 0.1,
    },
    2030: {
        'min_delay': 0.8,
        'load_leveling': 0.2,
    },
    2035: {
        'min_delay': 0.6,
        'load_leveling': 0.4,
    },
    2040: {
        'min_delay': 0.5,
        'load_leveling': 0.5,
    },
    2045: {
        'min_delay': 0.4,
        'load_leveling': 0.6,
    },
    2050: {
        'min_delay': 0.3,
        'load_leveling': 0.7,
    },
}

In [ ]:
# some files use state abbreviations and some use full names, so this dictionary can convert between them
STATE_ABBREVIATIONS_TO_NAMES = {
    'AB': 'alberta', 'AK': 'alaska', 'AL': 'alabama', 'AR': 'arkansas', 'AZ': 'arizona',
    'CA': 'california', 'BC': 'british columbia', 'MX': 'mexico',
    'CO': 'colorado', 'CT': 'connecticut', 'DC': 'district of columbia', 'DE': 'delaware',
    'FL': 'florida', 'GA': 'georgia', 'HI': 'hawaii', 'IA': 'iowa', 'ID': 'idaho', 'IL': 'illinois',
    'IN': 'indiana', 'KS': 'kansas', 'KY': 'kentucky', 'LA': 'louisiana', 'MA': 'massachusetts',
    'MD': 'maryland', 'ME': 'maine', 'MI': 'michigan', 'MN': 'minnesota', 'MO': 'missouri',
    'MS': 'mississippi', 'MT': 'montana', 'NC': 'north carolina', 'ND': 'north dakota',
    'NE': 'nebraska', 'NH': 'new hampshire', 'NJ': 'new jersey', 'NM': 'new mexico', 'NV': 'nevada',
    'NY': 'new york', 'OH': 'ohio', 'OK': 'oklahoma', 'OR': 'oregon', 'PA': 'pennsylvania',
    'RI': 'rhode island', 'SC': 'south carolina', 'SD': 'south dakota', 'TN': 'tennessee',
    'TX': 'texas', 'UT': 'utah', 'VA': 'virginia', 'VT': 'vermont', 'WA': 'washington',
    'WI': 'wisconsin', 'WV': 'west virginia', 'WY': 'wyoming'
}


In [ ]:
# list of all available scenarios
scenarios = ['BAU_Climate', 'NetZeroNoCCS_Climate']

# list of all available climates
climates = ['rcp45cooler', 'rcp85hotter']

# list of all available populations which correspond to the climates
# in other words, population should match the climate - ssp3:rcp45 and ssp5:rcp85
populations = ['ssp3', 'ssp5']

# list of all available years
years = [2040, 2045] #[2025, 2030, 2035, 2050]


In [ ]:
for scenario in scenarios:
    for i, climate in enumerate(climates):
        population = populations[i]
        for year in years:

            # load LDV min_delay files
            ldv_min_delay_files = sorted(glob(f'{path_to_ldv_output}/{scenario}/{climate}/min_delay/*{year}*.csv'))
            ldv_min_delay_data = pd.concat([pd.read_csv(f, parse_dates=['time']) for f in ldv_min_delay_files])

            # load LDV load_leveling files
            ldv_load_leveling_files = sorted(glob(f'{path_to_ldv_output}/{scenario}/{climate}/load_leveling/*{year}*.csv'))
            ldv_load_leveling_data = pd.concat([pd.read_csv(f, parse_dates=['time']) for f in ldv_load_leveling_files])

            # load MDV files but filter out already combined WECC files
            mdv_files = [f for f in sorted(glob(f'{path_to_mhdv_output}/*mdv*{scenario}*{year}*.csv')) if 'WECC' not in f]
            mdv_data = pd.concat([pd.read_csv(f, parse_dates=['time']) for f in mdv_files])
            mdv_data['time'] = mdv_data.time.dt.tz_localize('utc')

            # load HDV files but filter out already combined WECC files
            hdv_files = [f for f in sorted(glob(f'{path_to_mhdv_output}/*hdv*{scenario}*{year}*.csv')) if 'WECC' not in f]
            hdv_data = pd.concat([pd.read_csv(f, parse_dates=['time']) for f in hdv_files])
            hdv_data['time'] = hdv_data.time.dt.tz_localize('utc')

            # load rail, aviation, ship data
            rail_aviation_ship_data = pd.read_excel(path_to_rail_aviation_ship_data).fillna(0)

            # load county to BA mapping file
            ba_county_mapping = pd.read_csv(path_to_county_ba_mapping).rename(columns={'FIPS': 'County_FIPS', 'Balancing Authority': 'BA_Code'})

            # load county population file and filter to this year
            county_population = pd.read_csv(
                f'{path_to_county_population}/{population}_county_population.csv'
            )
            if str(year) not in county_population.columns:
                county_population[str(year)] = (county_population[str(year+5)] + county_population[str(year-5)]) / 2
            county_population = county_population[['FIPS', str(year)]].rename(columns={
                'FIPS': 'County_FIPS',
                str(year): 'Population',
            })
            county_population['Population'] = county_population['Population'].clip(0, None)

            # calculate the population share weight of each county to its state and each of its BAs
            ba_county_population = ba_county_mapping.merge(county_population, how='left', on='County_FIPS')
            ba_county_population['BA_Population_Sum'] = ba_county_population.groupby('BA_Code')['Population'].transform('sum')
            ba_county_population['BA_Population_Fraction'] = ba_county_population['Population'].values / ba_county_population['BA_Population_Sum'].values
            ba_county_population['County_Population_Fraction_Sum'] = ba_county_population.groupby('County_FIPS')['BA_Population_Fraction'].transform('sum')
            ba_county_population['Population_Weight'] = ba_county_population['BA_Population_Fraction'].values / ba_county_population['County_Population_Fraction_Sum'].values
            ba_county_population['State_Name'] = ba_county_population['State_Name'].str.lower()

            # merge the rail, aviation, and ship data with the population weight
            rail_aviation_ship_data_ba = rail_aviation_ship_data[[
                'FIPS',
                'County Name',
                'State Name',
                "County's share of total state passanger railroad",
                "County's share of total state freight railroad",
                "Percetage aviation",
                "County's share of total state dock",
            ]].rename(columns={
                'FIPS': 'County_FIPS',
                'County Name': 'County_Name',
                'State Name': 'State_Name',
                "County's share of total state passanger railroad": 'passenger_rail_share',
                "County's share of total state freight railroad": 'freight_rail_share',
                "Percetage aviation": 'aviation_share',
                "County's share of total state dock": "ship_share",
            }).merge(ba_county_population[['County_FIPS', 'BA_Code', 'Population_Weight']], how='left', on='County_FIPS').fillna(0)
            rail_aviation_ship_data_ba['State_Name'] = rail_aviation_ship_data_ba['State_Name'].str.lower()

            # read the GCAM transportation energy by state and convert to MW
            gcam_transportation_energy = pd.read_csv(f'{path_to_gcam_energy_output}/transportation_energy_output_godeeep_{scenario}.csv')
            gcam_transportation_energy = gcam_transportation_energy[gcam_transportation_energy['Year'] == year]
            gcam_transportation_energy = gcam_transportation_energy[gcam_transportation_energy['technology'].isin(['BEV', 'Electric'])]
            gcam_transportation_energy = gcam_transportation_energy[gcam_transportation_energy['region'].isin(STATE_ABBREVIATIONS_TO_NAMES.keys())]
            gcam_transportation_energy['State_Name'] = gcam_transportation_energy['region'].replace(STATE_ABBREVIATIONS_TO_NAMES)
            gcam_transportation_energy['mw'] = gcam_transportation_energy['value'] * 277.77777777778 * 1000000 / 8760

            # for each of rail, aviation, and ship, calculate the BA level load using the population weights and county shares
            rail_aviation_ship_data_ba['passenger_rail_load_MWh'] = rail_aviation_ship_data_ba['passenger_rail_share'] * rail_aviation_ship_data_ba['Population_Weight'] * rail_aviation_ship_data_ba.merge(
                gcam_transportation_energy[gcam_transportation_energy['subsector'].isin([
                    'HSR', 'Passenger Rail'
                ])][['State_Name', 'mw']].groupby('State_Name').sum().reset_index(),
                how='left',
                on='State_Name',
            )['mw']

            rail_aviation_ship_data_ba['freight_rail_load_MWh'] = rail_aviation_ship_data_ba['freight_rail_share'] * rail_aviation_ship_data_ba['Population_Weight'] * rail_aviation_ship_data_ba.merge(
                gcam_transportation_energy[gcam_transportation_energy['subsector'].isin([
                    'Freight Rail'
                ])][['State_Name', 'mw']].groupby('State_Name').sum().reset_index(),
                how='left',
                on='State_Name',
            )['mw']

            rail_aviation_ship_data_ba['aviation_load_MWh'] = rail_aviation_ship_data_ba['aviation_share'] * rail_aviation_ship_data_ba['Population_Weight'] * rail_aviation_ship_data_ba.merge(
                gcam_transportation_energy[gcam_transportation_energy['subsector'].isin([
                    'International Aviation', 'Domestic Aviation'
                ])][['State_Name', 'mw']].groupby('State_Name').sum().reset_index(),
                how='left',
                on='State_Name',
            )['mw']

            rail_aviation_ship_data_ba['ship_load_MWh'] = rail_aviation_ship_data_ba['ship_share'] * rail_aviation_ship_data_ba['Population_Weight'] * rail_aviation_ship_data_ba.merge(
                gcam_transportation_energy[gcam_transportation_energy['subsector'].isin([
                    'International Ship', 'Domestic Ship'
                ])][['State_Name', 'mw']].groupby('State_Name').sum().reset_index(),
                how='left',
                on='State_Name',
            )['mw']

            # Group loads by BA and sum
            ba_rail_ship_aviation_mw = rail_aviation_ship_data_ba.groupby('BA_Code')[['passenger_rail_load_MWh', 'freight_rail_load_MWh', 'aviation_load_MWh', 'ship_load_MWh']].sum().reset_index()

            # sanity check the WECC level loads 
            print(ba_rail_ship_aviation_mw.sum())

            # combining LDV charging strategies based on the share
            ldv_data = ldv_load_leveling_data.rename(columns={'load_MWh': 'load_leveling_MWh'}).merge(
                ldv_min_delay_data.rename(columns={'load_MWh': 'min_delay_MWh'}),
                how='outer',
                on=['time', 'balancing_authority'],
            )
            ldv_data['load_MWh'] = ldv_data.min_delay_MWh * LDV_CHARGING_SHARE_BY_YEAR[year]['min_delay'] + ldv_data.load_leveling_MWh * LDV_CHARGING_SHARE_BY_YEAR[year]['load_leveling']

            # scale ldv shapes to match gcam
            # NOTE: this is now handled during LDV processing in the other notebook
            gcam_ldv = gcam_transportation_energy[gcam_transportation_energy['subsector'].isin([
                '2W and 3W',
                'Car',
                'Large Car and Truck',
                'Light truck',
            ]) & gcam_transportation_energy.region.isin([
                'CA','OR','WA','AZ','NV','WY','ID','UT','NM','CO','MT'
            ])].mw.sum() * 8760
            # scale_factor = gcam_ldv / ldv_data['load_MWh'].sum()
            # ldv_data['load_MWh'] = ldv_data['load_MWh'] * scale_factor

            # combine the LDV, MDV, HDV, and rail, aviation, ship loads,
            # clamping mhdv to the 8760 UTC year
            transportation_hourly_load = ldv_data[['time', 'balancing_authority', 'load_MWh']].rename(columns={
                'load_MWh': 'LDV_load_MWh'
            }).merge(
                mdv_data[
                    (mdv_data.time > f'{year}-01-01') & 
                    (mdv_data.time <= f'{year+1}-01-01')
                ].rename(columns={
                    'MW': 'MDV_load_MWh',
                    'BA': 'balancing_authority',
                }),
                how='outer',
                on=['balancing_authority', 'time'],
            ).merge(
                hdv_data[
                    (hdv_data.time > f'{year}-01-01') & 
                    (hdv_data.time <= f'{year+1}-01-01')
                ].rename(columns={
                    'MW': 'HDV_load_MWh',
                    'BA': 'balancing_authority',
                }),
                how='outer',
                on=['balancing_authority', 'time'],
            ).merge(
                ba_rail_ship_aviation_mw.rename(columns={
                    'BA_Code': 'balancing_authority'
                }),
                how='outer',
                on='balancing_authority',
            )
            transportation_hourly_load['transportation_load_MWh'] = transportation_hourly_load[[
                'LDV_load_MWh', 'MDV_load_MWh', 'HDV_load_MWh', 'passenger_rail_load_MWh', 'freight_rail_load_MWh', 'aviation_load_MWh', 'ship_load_MWh'
            ]].sum(axis=1)
            transportation_hourly_load = transportation_hourly_load[transportation_hourly_load.time.notna()]

            # transportation_hourly_load.transportation_load_MWh.sum() / (gcam_transportation_energy[gcam_transportation_energy.region.isin([
            #     'CA','OR','WA','AZ','NV','WY','ID','UT','NM','CO','MT'
            # ])].mw.sum() * 8760)

            # transportation_hourly_load.LDV_load_MWh.sum(), gcam_ldv

            # write out csv files per balancing authority
            for group, data in transportation_hourly_load.groupby('balancing_authority'):
                out_path = Path(f"{base_path}/output/{scenario}/{climate}/{group}_hourly_transportation_load_{scenario}_{climate}_{year}.csv")
                out_path.parent.mkdir(parents=True, exist_ok=True)
                data.to_csv(out_path, index=False)


In [ ]:
# plot a single BA as a sanity check
fig, ax = plt.subplots()
transportation_hourly_load[transportation_hourly_load.balancing_authority == 'WWA'].drop(columns='balancing_authority').set_index('time').resample(
    '1D',
    label='left',
).sum().plot.area(
    #x='time',
    ax=ax,
    y=['passenger_rail_load_MWh', 'freight_rail_load_MWh', 'aviation_load_MWh', 'ship_load_MWh', 'HDV_load_MWh', 'MDV_load_MWh', 'LDV_load_MWh', ]
    #y='transportation_load_MWh'
)
ax.set_title('WWA Transportation Load, NetZeroNoCCS_Climate')
ax.set_ylabel('MWh')

In [ ]:
# this loop generates WECC level combined loads
for scenario in scenarios:
    for climate in climates:
        for year in years:
            data = pd.concat([pd.read_csv(f) for f in glob(
                f"{base_path}/output/{scenario}/{climate}/*_hourly_transportation_load_{scenario}_{climate}_{year}.csv"
            )])
            wecc = data.groupby('time')[[
                'passenger_rail_load_MWh', 'freight_rail_load_MWh', 'aviation_load_MWh', 'ship_load_MWh', 'HDV_load_MWh', 'MDV_load_MWh', 'LDV_load_MWh', 'transportation_load_MWh'
            ]].sum().reset_index()
            wecc.to_csv(
                f"{base_path}/output/wecc_hourly_transportation_load_{scenario}_{climate}_{year}.csv"
            )
            

In [ ]:
# sanity checks

In [ ]:
scenario, climate, year

In [ ]:
wecc['time'] = pd.to_datetime(wecc.time)

In [ ]:
fig, ax = plt.subplots()
wecc.set_index('time').resample(
    '1D',
    label='left',
).mean().plot.area(
    ax=ax,
    y=['passenger_rail_load_MWh', 'freight_rail_load_MWh', 'aviation_load_MWh', 'ship_load_MWh', 'HDV_load_MWh', 'MDV_load_MWh', 'LDV_load_MWh', ],
    cmap='turbo_r'
)
ax.set_title('WECC Transportation Load, NetZeroNoCCS_Climate')
ax.set_ylabel('MWh')

In [ ]:
fig, ax = plt.subplots()
wecc.set_index('time').resample(
    '1D',
    label='left',
).max().plot.line(
    ax=ax,
    y='transportation_load_MWh',
)
ax.set_title('WECC Peak Transportation Load, NetZeroNoCCS_Climate')
ax.set_ylabel('MWh')